# DeepLabCut for single animal tracking

Some useful links:

- [DeepLabCut's GitHub: github.com/DeepLabCut/DeepLabCut](https://github.com/DeepLabCut/DeepLabCut)
- [DeepLabCut's Documentation: User Guide for Single Animal projects](https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html)

This notebook illustrates how to use the cloud to
- label data on DeepLabCut GUI
- create training set
- train a network
- evaluate a network
- analyze novel videos

**This notebook is for user who do not have GPU computer.**

Original Demo Notebook: https://deeplabcut.github.io/DeepLabCut/examples/COLAB/COLAB_DEMO_mouse_openfield.html

*Modified by Do-young on 26.01.13*

# Step 1: Label Data on DeepLabCut GUI
1. Install DeepLabCut GUI
Open Anaconda Prompt
```python
conda create -n dlc python=3.9 -y
conda activate dlc
pip install "deeplabcut[gui]"
pip install "deeplabcut[tf]"
python -m deeplabcut
```
2. Create a new project
- Write 'project name' and 'Experimenter'
- Select folder that contains your videos and choose only 3-5 videos in the list.
- Click 'Copy videos to project folder'
3. Open `config.yaml` and fix the following part:
```
bodyparts:
- bodypart1
- bodypart2
- bodypart3
- objectA
```
For mouse with LED on its head, remove all the bodyparts and add LED
4. Extract frames
- Click 'Extract frames' banner
- Keep the default setting of 'Attributes'
- Change the video format to match with your videos
5. Label frames
- Click 'Label frames' banner
- Click 'Label frames' button
- Select folder with each videos one by one

Then you will see another window named 'napari'

- Adjust the point size
- Click the XOR button
- Click on the LED position
- Repeat for every frame
- Save (Ctrl+S)


# Step 2: Colab Environment Setting
##1. Change Runtime Type
- Python3
- GPU


## 2. Install DeepLabCut and TensorFlow

In [ ]:
!pip install --pre deeplabcut

In [ ]:
import deeplabcut

## 3. Link your Google Drive
- Make sure to upload the **project folder** of DeepLabCut that we stored the labels & **folder** with whole videos.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# PLEASE EDIT THIS:
project_folder_name = "project_folder_name"
video_type = "avi" #, mp4, MOV, or avi, whatever you uploaded!

# No need to edit this, we are going to assume you put videos you want to analyze
# in the "videos" folder, but if this is NOT true, edit below:
videofile_path = [f"/content/drive/My Drive/{project_folder_name}/videos/"]
print(videofile_path)

# The prediction files and labeled videos will be saved in this `labeled-videos` folder
# in your project folder; if you want them elsewhere, you can edit this;
# if you want the output files in the same folder as the videos, set this to an empty string.
destfolder = f"/content/drive/My Drive/{project_folder_name}/labeled-videos"

#No need to edit this, as you set it when you passed the ProjectFolderName (above):
path_config_file = f"/content/drive/My Drive/{project_folder_name}/config.yaml"
print(path_config_file)

# This creates a path variable that links to your Google Drive project

# Step 3: Training
## 1. Create a training dataset

In [ ]:
deeplabcut.create_training_dataset(
  path_config_file, net_type="resnet_50", engine=deeplabcut.Engine.PYTORCH
)

## 2. Training
- Even though the 'Model performance' looks like this:
```
Model performance:
  metrics/test.rmse:            nan
  metrics/test.rmse_pcutoff:    nan
  metrics/test.mAP:            0.00
  metrics/test.mAR:            0.00
```
Don't worry, it's working well.

In [ ]:
# Let's also change the display and save_epochs just in case Colab takes away
# the GPU... If that happens, you can reload from a saved point using the
# `snapshot_path` argument to `deeplabcut.train_network`:
#   deeplabcut.train_network(..., snapshot_path="/content/.../snapshot-050.pt")

# Typically, you want to train to ~200 epochs. We set the batch size to 8 to
# utilize the GPU's capabilities.

# More info and there are more things you can set:
#   https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html#g-train-the-network

deeplabcut.train_network(
    path_config_file,
    shuffle=1,
    save_epochs=5,
    epochs=200,
    batch_size=8,
)

## 3. Evaluating
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .csv file in a subdirectory under **evaluation-results-pytorch**

In [ ]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as
# good as the labeler, so be sure your labels are good!

# Step 4: Analyze Videos (Model Application)
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides.

In [ ]:
import glob
import os

video_folder_path = '/content/drive/MyDrive/My_WebCam'

all_videos = glob.glob(os.path.join(video_folder_path, '*.avi'))
all_videos.sort()

print(f" {len(all_videos)} videos found")

deeplabcut.analyze_videos(path_config_file, all_videos, save_as_csv=True)

#outlier processing
deeplabcut.filterpredictions(path_config_file, all_videos, save_as_csv=True)


Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Merge csv files of each video
- In the directory where the video resides, csv files that contains LED x_coordinate & y_coordinate are saved.
- To generate the whole time scale position infomation csv file, run the following code

In [ ]:
import pandas as pd
import glob
import os
import re

# 1. Edit path of folder where csv files are saved
csv_folder_path = '/content/drive/MyDrive/video_folder/'

# 2. Find desired csv files (filtered version)
file_pattern = os.path.join(csv_folder_path, '*_filtered.csv')
all_files = glob.glob(file_pattern)

# 3. Sort the csv in ascending order
def get_leading_number(filepath):
    filename = os.path.basename(filepath)
    match = re.match(r'^(\d+)', filename)
    return int(match.group(1)) if match else -1

all_files.sort(key=get_leading_number)

print(f"Total {len(all_files)}files found.")
print(f"First csv file: {os.path.basename(all_files[0])}")
print(f"Last csv file: {os.path.basename(all_files[-1])}")

# 4. Concatenate
df_list = []

for file in all_files:
    # DLC CSV header: 3 column
    # DLC CSV 1st column: frame index
    df = pd.read_csv(file, header=[0, 1, 2], index_col=0)
    df_list.append(df)

merged_df = pd.concat(df_list)

# frame index adjustment
merged_df = merged_df.reset_index(drop=True)

# 5. Save file
output_path = os.path.join(csv_folder_path, 'Final_Merged_Result.csv')
merged_df.to_csv(output_path)

print("-" * 30)
print(f"Merge Completed. Saved in {output_path}")
print(f"Check the timeStamp.csv whether its index match with{len(merged_df)}")

# Step 5: Create Labeled video (Optional)
This function is for visualization purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
deeplabcut.create_labeled_video(path_config_file, all_videos, filtered=True)